Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Required libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import callbacks
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv1D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

Read CSV files (Energy consumption dataset)

In [ ]:
# data = pd.read_csv('/content/drive/MyDrive/10_Microwave.csv')
# data = pd.read_csv('/content/drive/MyDrive/apartment-electrical/apartment/2016/Apt1_2016.csv')
data = pd.read_csv('/content/drive/MyDrive/Apt1_2016.csv')
data.shape

(503774, 2)

Printing first few rows to check data availablity

In [ ]:
# Rename columns in Apt1_data
data = data.rename(columns={'2016-01-01 00:00:00': 'timestamp'})
data = data.rename(columns={'0.68415': 'power'})
print(data.head())

             timestamp     power
0  2016-01-01 00:01:00  0.682950
1  2016-01-01 00:02:00  0.682983
2  2016-01-01 00:03:00  0.682383
3  2016-01-01 00:04:00  0.682283
4  2016-01-01 00:05:00  0.682017


In [ ]:
# # Extract the first 10,000 values
# timestamps = data['timestamp'].head(10000).tolist()
# powers = data['power'].head(10000).tolist()

# # Plotting
# plt.plot(timestamps, powers)
# plt.xlabel('Timestamp')
# plt.ylabel('Power')
# plt.title('Power vs Timestamp Plot (First 10,000 values)')
# plt.show()

Adding labels to daata

In [ ]:
power = data['power']
data['label'] = pd.Series([0 if value < 0.80 else (1 if value < 3 else 2) for value in power])
data

,timestamp,power,label
0,2016-01-01 00:01:00,0.682950,0
1,2016-01-01 00:02:00,0.682983,0
2,2016-01-01 00:03:00,0.682383,0
3,2016-01-01 00:04:00,0.682283,0
4,2016-01-01 00:05:00,0.682017,0
...,...,...,...
503769,2016-12-15 20:10:00,0.634417,0
503770,2016-12-15 20:11:00,1.914167,1
503771,2016-12-15 20:12:00,3.837700,2
503772,2016-12-15 20:13:00,4.717600,2


Processing/Cleaning data

In [ ]:
data = data.dropna()

#X = data[['year', 'month', 'day', 'hour']]
X = data['power']
# X = data[['hour']]
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Adding callback

In [ ]:
from tensorflow.keras import callbacks

myCallback = callbacks.ReduceLROnPlateau(
             monitor = 'val_loss',
             patience = 3,
             factor = 0.02,
             lr = 1e-30
)

Generating an ANN model

In [ ]:
model = Sequential([
    Dense(2056, activation="relu"),
    Dense(1024, activation="relu"),
    BatchNormalization(),
    Dropout(0.5),

    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    BatchNormalization(),
    Dropout(0.5),

    Dense(3, activation='softmax')  # Output layer for three classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'], run_eagerly = True)

Training the model

In [ ]:
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test), callbacks=[myCallback], verbose = 1)

NameError: name 'model' is not defined

Visual representation of training loss vs validation loss / training accuracy vs validation accuracy

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

# Retrieve a list of list results on training and test data sets for each training epoch
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))  # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, color='green', label='Training Accuracy')  # Change color to green
plt.plot(epochs, val_acc, color='orange', label='Validation Accuracy')  # Change color to orange
plt.title('Training and validation accuracy')
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, color='blue', label='Training Loss')  # Change color to blue
plt.plot(epochs, val_loss, color='red', label='Validation Loss')  # Change color to red
plt.title('Training and validation loss')
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.figure()


#------------------------------------------------
# Plot histograms of accuracy and loss
#------------------------------------------------
plt.figure(figsize=(12, 4))

# Accuracy histogram
plt.subplot(1, 2, 1)
plt.hist(acc, bins=20, color='blue', alpha=0.7, label='Training Accuracy')
plt.hist(val_acc, bins=20, color='orange', alpha=0.7, label='Validation Accuracy')
plt.title('Accuracy Histogram')
plt.legend()

# Loss histogram
plt.subplot(1, 2, 2)
plt.hist(loss, bins=20, color='blue', alpha=0.7, label='Training Loss')
plt.hist(val_loss, bins=20, color='orange', alpha=0.7, label='Validation Loss')
plt.title('Loss Histogram')
plt.legend()

plt.show()

Saving model in google drive's path

In [ ]:
model.save('/content/drive/MyDrive/EnergyModel.h5')